In [ ]:
import sys
import os
# Add the parent directory (where 'src' folder is located) to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from src.stock_features import create_target_variable
from strategies.continuous_allocation import ContinuousAllocationStrategy
from strategies.MLDynamicStrategy import MLDynamicStrategy


In [ ]:
data_path = r'C:\Users\epoch_bpjmdqk\Documents\Code\data\processed\stock_and_macro.csv'
# Path to the saved model
model_path = r'C:\Users\epoch_bpjmdqk\Documents\Code\models\randomforest_w10_t0.005.pkl'
# Define the data preparation parameters that the model was trained on
window = 10
# Define the backtesting period split
backtest_start_date = '2021-01-01'
# NEW: Define real-world trading costs and starting balance
transaction_cost_per_trade = 0.001
slippage_factor = 0.0005
starting_balance = 10000.0
allocation_percentage = 0.50

target_ticker = 'WMT'
threshold = 0.005

In [ ]:
try:
    # Load the processed data
    data = pd.read_csv(data_path, index_col='Date', parse_dates=True)
    
    # Load the trained model from the pickle file
    with open(model_path, 'rb') as file:
        best_model = pickle.load(file)
    
    print("✅ Successfully loaded data and the best model.")
except FileNotFoundError:
    print(f"Error: Could not find one of the files. Please check paths:")
    print(f"Data path: {data_path}")
    print(f"Model path: {model_path}")
    exit()

In [ ]:
# ================================================================
# Step 3: Your Data Preparation
# ================================================================

# Create the target variable
data_with_target = create_target_variable(data.copy(), target_ticker, window=window, threshold=threshold)

# Define column names
target_col_name = f'{target_ticker}_Target'
target_return_col_name = f'{target_ticker}_target_return_{window}D_{threshold}'
columns_to_drop = [
    target_col_name,
    target_return_col_name,
    f'Open_{target_ticker}',
    f'High_{target_ticker}',
    f'Low_{target_ticker}',
    f'Close_{target_ticker}'
] 
data_with_target.dropna(inplace=True)

# Split data into training and backtesting sets for features
# NOTE: We use the *entire* dataset here to generate all predictions,
# but we will only backtest on the backtest_start_date onwards.
X_full = data_with_target.drop(columns=columns_to_drop)


# Generate Predictions

# Use the loaded model to predict signals on your entire feature set.
# This creates the 'Prediction' column for the full DataFrame.
data_with_target['Prediction'] = best_model.predict(X_full)
print("✅ Successfully generated predictions using the loaded model.")

# Prepare OHLCV DataFrame for backtesting.py
ohlc = data_with_target.loc[backtest_start_date:, [
    f'Open_{target_ticker}',
    f'High_{target_ticker}',
    f'Low_{target_ticker}',
    f'Close_{target_ticker}',
    f'Volume_{target_ticker}',
    'Prediction'
]].rename(columns={
    f'Open_{target_ticker}': 'Open',
    f'High_{target_ticker}': 'High',
    f'Low_{target_ticker}': 'Low',
    f'Close_{target_ticker}': 'Close',
    f'Volume_{target_ticker}': 'Volume'
})

In [ ]:
print(ohlc['Prediction'].value_counts())

In [ ]:
# ================================================================
# Step 4: Run the Backtest with the Predictions
# ================================================================

bt = Backtest(
    ohlc,  # Start the backtest from the specified date
    ContinuousAllocationStrategy,
    cash=starting_balance,
    commission=transaction_cost_per_trade,
) 


print("\n--- Running Optimization ---")
stats = bt.optimize(
    allocation_percentage=list(np.arange(0.1, 1.01, 0.1)),
    sl_percentage=list(np.arange(0.01, 0.1, 0.01)),
    #tp_percentage=list(np.arange(0.05, 0.25, 0.05)),
    maximize='Sharpe Ratio',
)

In [ ]:
# Print the best results and the corresponding parameters
print("\n--- Best Backtest Results ---")
print(stats._strategy)
print(stats)